In [1]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=110,
    target_version=black.TargetVersion.PY310,
)

In [2]:
import numpy as np
import lysis
import os

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FFMpegWriter, FuncAnimation
from matplotlib.colors import ListedColormap

from IPython.display import HTML

plt.rcParams["animation.html"] = "jshtml"

In [3]:
e = lysis.util.Experiment(os.path.join("..", "..", "data"), experiment_code="2023-01-30-2005")
e.read_file()
file_code = "_PLG2_tPA01_Kd00020036_along_Q2.dat"

In [4]:
print(e)

experiment_code : 2023-01-30-2005
data_filenames  : unbinding_time    : tsectPA.dat
                  lysis_time        : lysismat.dat
                  total_lyses       : lenlysisvect.dat
                  degradation_state : deg.dat
micro_params    : None
macro_params    : binding_rate       : 0.1
                  pore_size          : 0.00010135
                  diffusion_coeff    : 5e-07
                  binding_sites      : 427
                  forced_unbind      : 0.5143
                  average_bind_time  : 277.8
                  grid_node_distance : 1.0862
                  cols               : 93
                  rows               : 121
                  full_row           : 278
                  xz_row             : 185
                  total_edges        : 33,545
                  total_fibers       : 25,761
                  empty_rows         : 28
                  last_empty_edge    : 7,783
                  total_molecules    : 43,074
                  moving_pr

In [5]:
deg = np.fromfile(os.path.join(e.os_path, "deg" + file_code))
tsave = np.fromfile(os.path.join(e.os_path, "tsave" + file_code))

In [6]:
deg = deg.reshape(e.macro_params.total_trials, e.macro_params.number_of_saves, e.macro_params.total_edges)

In [7]:
tsave = tsave.reshape(e.macro_params.total_trials, e.macro_params.number_of_saves)

In [8]:
mapped_deg = deg
# mapped_deg[:,:,e.macro_params.last_empty_edge+1:][deg[:,:,e.macro_params.last_empty_edge+1:] == 0] = e.macro_params.total_time * 10
mapped_deg = -deg
mapped_deg[deg == 0] = e.macro_params.total_time * 2
mapped_deg[deg == -1] = 0
# for i in range(e.macro_params.total_edges):
#    mapped_deg[:, :, i] -= tsave

In [9]:
def plot_coords(i, j):
    x = j
    y = i
    if j % 3 == 0:
        return x / 3.0, y + 0.5
    if j % 3 == 1:
        return (x - 1) / 3.0, y
    if j % 3 == 2:
        return (x - 2) / 3.0 + 0.5, y

In [10]:
x = np.empty(e.macro_params.total_edges, dtype=float)
y = np.empty(e.macro_params.total_edges, dtype=float)
for k in range(e.macro_params.total_edges):
    i, j = lysis.from_fortran_edge_index(k, e.macro_params.rows, e.macro_params.cols)
    x[k], y[k] = plot_coords(i, j)

In [11]:
colormap = plt.cm.viridis(np.arange(plt.cm.viridis.N))
colormap[:, 3] = 0.75
colormap = ListedColormap(colormap)

# colormap = mpl.colormaps['viridis']
# colormap[:,0:3] *= 0.5
colormap.set_extremes(bad="k", under="w", over="k")

In [12]:
run = 1

plt.ioff()
fig = plt.figure(figsize=(7, 7))
ax = fig.add_axes([0, 0, 1, 1])  # , frameon=False)
vmin = 0
vmax = e.macro_params.cols - 1

ydiff = e.macro_params.rows - e.macro_params.cols

ax.set_xlim(vmin - 0.25, vmax + 0.25)
ax.set_ylim(vmin - 0.25, vmax + ydiff + 0.25)
ax.set_aspect("equal")
fig.canvas.draw()
s = (ax.get_window_extent().width / (vmax - vmin + 0.5) * 72 / (2 * fig.dpi)) ** 2
scatt = ax.scatter(
    x,
    y,
    s=s,
    marker="s",
    linewidths=0,
    c=mapped_deg[run, 0],
    cmap=colormap,
    # vmin = 0, vmax=e.macro_params.total_time,
    plotnonfinite=True,
    vmin=1,
    vmax=mapped_deg[run, e.macro_params.number_of_saves - 1][
        mapped_deg[run, e.macro_params.number_of_saves - 1] < e.macro_params.total_time
    ].max(),
)


def update(frame_number):
    scatt.set_array(mapped_deg[run, frame_number])


animation = FuncAnimation(fig, update, frames=np.arange(e.macro_params.number_of_saves), interval=200)

FFwriter = FFMpegWriter(fps=10)
animation.save(os.path.join(e.os_path, f"deg_animation_{run:02}" + file_code[:-4] + ".mp4"), writer=FFwriter)
plt.close()

# HTML(animation.to_jshtml())
# HTML(animation.to_html5_video())